In [87]:
import numpy as np
import scipy 
from scipy.fft import fft
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [88]:
def generate_signal(timestep, numsamples):
    """
    Generate a sine wave signal.
    
    Parameters:
    - timestep (float): Time step between each sample.
    - numsamples (int): Number of samples to generate.
    
    Returns:
    - np.array: Sine wave signal of specified frequency (40Hz).
    """
    
    # Generate a time vector ranging from 0 to (numsamples * timestep)
    t = np.linspace(0.0, numsamples*timestep, numsamples, endpoint=False)
    
    # Compute the sine wave signal for the given time vector
    signal = np.sin(40.0 * 2.0 * np.pi * t)  # This will generate a 40Hz sine wave
    return signal

def fft_calculate(data, timestep):
    """
    Calculate the one-sided FFT (Fast Fourier Transform) of the given data.
    
    Parameters:
    - data (np.array): Time-domain signal data.
    - timestep (float): Time step between each sample of the data.
    
    Returns:
    - tuple: Frequency array (xf) and normalized magnitude array of the FFT (yf).
    """
    
    # Compute the magnitude of the FFT of the data
    yf = np.abs(fft(data))
    
    # Get the number of samples in the data
    numsamples = len(data)
    
    # Compute the corresponding frequency values for the FFT result
    xf = np.fft.fftfreq(numsamples, timestep)[:numsamples//2]
    
    # Return frequency values and normalized magnitude of the FFT
    return xf, yf[0:numsamples//2] * 2.0 / numsamples

def find_nearest(array, value):
    """
    Find the nearest value and its index in an array to the specified value.
    
    Parameters:
    - array (list or np.array): Array of values to search within.
    - value (float): Value to find the nearest match to.
    
    Returns:
    - tuple: Index of the nearest value and the nearest value itself.
    """
    
    # Convert the input array to a numpy array for efficient computations
    array = np.asarray(array)
    
    # Find the index of the nearest value in the array to the specified value
    idx = (np.abs(array - value)).argmin()
    
    # Return the index and the value from the array at that index
    return idx, array[idx]


The Simpson's rule for integration is given by:

$$
\int_{{x_{2i}}}^{{x_{2i+2}}} f(x) \, dx \approx \frac{h}{3} \left[ f(x_{2i}) + 4f(x_{2i+1}) + f(x_{2i+2}) \right]
$$

In [89]:
# def simpsons_integration(xf, yf, idx_start, idx_stop):
#     return scipy.integrate.simps(yf[idx_start:idx_stop], x=xf[idx_start:idx_stop])

def simpsons_integration(xf, yf, idx_start, idx_stop):
    n = idx_stop - idx_start
    if (n + 1) % 2 != 0:  # This should be not equal, since we want an odd number of points
        raise ValueError("Number of intervals should be even for Simpson's rule.")

    h = (xf[idx_stop-1] - xf[idx_start]) / n
    result = 0

    for i in range(n+1):  # Change here
        y = yf[idx_start + i]
        
        if i == 0 or i == n:
            result += y
        elif i % 2 == 0:
            result += 2*y
        else:
            result += 4*y

    result *= h/3
    return result

In [90]:
def visualize_signal_and_fft_simpsons(signal, timestep, mod_freq_hz, channel_separation_hz):
    xf, yf = fft_calculate(signal, timestep)

    freq_start = mod_freq_hz - channel_separation_hz / 2
    freq_stop = mod_freq_hz + channel_separation_hz / 2

    idx_start, _ = find_nearest(xf, freq_start)
    idx_stop, _ = find_nearest(xf, freq_stop)

    # Ensure even number of intervals
    if (idx_stop - idx_start) % 2 == 0:
        idx_stop += 1

    integrated_area = simpsons_integration(xf, yf, idx_start, idx_stop)

    
    fig = make_subplots(rows=2, cols=1, subplot_titles=("Time-domain Signal", f"FFT Magnitude - Integrated Simpsons Area: {integrated_area:.4f}"))
    
    # Time-domain Signal plot
    fig.add_trace(go.Scatter(y=signal, mode='lines', name='Time-domain Signal'), row=1, col=1)

    # FFT Magnitude plot
    fig.add_trace(go.Scatter(x=xf, y=yf, mode='lines', name='FFT Magnitude'), row=2, col=1)

    freq_start = mod_freq_hz - (channel_separation_hz * 0.25)
    freq_stop = mod_freq_hz + (channel_separation_hz * 0.25)
    idx_start, _ = find_nearest(xf, freq_start)
    idx_stop, _ = find_nearest(xf, freq_stop)

    fig.add_trace(go.Scatter(x=xf[idx_start:idx_stop], y=yf[idx_start:idx_stop], fill='tozeroy', fillcolor='rgba(127, 127, 127, 0.2)', name='Area of Interest'), row=2, col=1)

    fig.update_layout(title="Signal Analysis")

    fig.show()

In [91]:
if __name__ == "__main__":
    TIMESTEP = 0.01
    NUMSAMPLES = 1000
    MOD_FREQ_HZ = 40
    CHANNEL_SEPARATION_HZ = 40

    signal = generate_signal(TIMESTEP, NUMSAMPLES)
    visualize_signal_and_fft_simpsons(signal, TIMESTEP, MOD_FREQ_HZ, CHANNEL_SEPARATION_HZ)